In [1]:
import os
import cv2

In [2]:
from parking_position_detection_utils import ImageInfo
from parking_position_detection_utils import ImageProcessing

In [3]:
def draw_centroids_in_image(centroids, intersections, image):
    # Dibuja las intersecciones en color azul
    for point in intersections:
        cv2.circle(image, (int(point[0]), int(point[1])), radius=5, color=(255, 0, 0), thickness=-1)

    # Dibuja los centroides en color rojo con una 'x'
    for point in centroids:
        cv2.drawMarker(image, (int(point[0]), int(point[1])), color=(0, 0, 255), markerType=cv2.MARKER_TILTED_CROSS, markerSize=10, thickness=2)
    
    return image

In [4]:
def load_tagged_images(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            route = os.path.join(directory, filename)
            images.append(ImageInfo(route))
    return images


images_info = load_tagged_images('../parking_sequence')

In [5]:
images_info.sort(key=lambda x: x.time, reverse=False)
# for image in images_info:
#     print(image.distances, image.angles, image.time)

In [6]:
cv2.namedWindow("Lane", cv2.WINDOW_NORMAL)
for i in range(1):  # Repeat 1 time
    for idx in range(len(images_info)):
        image = cv2.imread(images_info[idx].image_path, cv2.IMREAD_COLOR)
        cv2.imshow("Lane", image)
        cv2.waitKey(100)
cv2.destroyAllWindows()

In [7]:
cv2.namedWindow("Centroids", cv2.WINDOW_NORMAL)
for i in range(1):  # Repeat 1 time
    for idx in range(len(images_info)):
        
        image = cv2.imread(images_info[idx].image_path, cv2.IMREAD_COLOR)
        imageGray = cv2.imread(images_info[idx].image_path, cv2.IMREAD_GRAYSCALE)

        image_processing = ImageProcessing(imageGray)  # Instantiate the class
        centroids, intersections = image_processing.get_vanishing_points()  # Call the method
        
        image_with_centroids = draw_centroids_in_image(centroids, intersections, image.copy())
        
        cv2.imshow("Centroids", image_with_centroids)
        cv2.waitKey(100)
        
cv2.destroyAllWindows()